# Comparação de Modelos de ML

* Bibliotecas:

In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score
import joblib
from pathlib import Path

* Leitura dos dados:

In [59]:
df= pd.read_csv("../data/raw/wines_atualizado.csv")

In [60]:
df_path = Path('../data/external/dictionary_atualizado.csv')
# data_path = Path('dictionary.csv')
df_dict = pd.read_csv(df_path, sep=',') 

In [61]:
df.head()

,vinícola,vinho,ano,avaliação,num_avaliações,região,preço,tipo,corpo,acidez,Categoria
0,Teso La Monja,Tinto,2013.0,4.9,58,Toro,995.00,Toro Red,5,3,Vinhos Tintos
1,Artadi,Vina El Pison,2018.0,4.9,31,Vino de Espana,313.50,Tempranillo,4,2,Vinhos Tintos
2,Vega Sicilia,Unico,2009.0,4.8,1793,Ribera del Duero,324.95,Ribera Del Duero Red,5,3,Vinhos Tintos
3,Vega Sicilia,Unico,1999.0,4.8,1705,Ribera del Duero,692.96,Ribera Del Duero Red,5,3,Vinhos Tintos
4,Vega Sicilia,Unico,1996.0,4.8,1309,Ribera del Duero,778.06,Ribera Del Duero Red,5,3,Vinhos Tintos


In [62]:
df_dict.head(20)

,variavel,descricao,tipo,subtipo
0,vinícola,Nome da vinícola,qualitativa,nominal
1,vinho,Nome do vinho,qualitativa,nominal
2,ano,Ano em que as uvas foram colhidas,quantitativa,discreta
3,avaliação,Avaliação média dada ao vinho pelos usuários [...,quantitativa,contínua
4,num_avaliações,Número de usuários que avaliaram o vinho,quantitativa,discreta
5,região,Região do vinho,qualitativa,nominal
6,preço,Preço em euros [€],quantitativa,contínua
7,tipo,Variedade de vinho,qualitativa,nominal
8,corpo,"Pontuação de corpo, definida como a riqueza e ...",qualitativa,ordinal
9,acidez,"Pontuação de acidez, definida como o frescor e...",qualitativa,ordinal


* Verificando estrutura:

In [63]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2048 entries, 0 to 2047
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   vinícola        2048 non-null   object 
 1   vinho           2048 non-null   object 
 2   ano             1976 non-null   float64
 3   avaliação       2048 non-null   float64
 4   num_avaliações  2048 non-null   int64  
 5   região          2048 non-null   object 
 6   preço           2048 non-null   float64
 7   tipo            1942 non-null   object 
 8   corpo           2048 non-null   int64  
 9   acidez          2048 non-null   int64  
 10  Categoria       2048 non-null   object 
dtypes: float64(3), int64(3), object(5)
memory usage: 176.1+ KB


Valores numéricos:

In [64]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
ano,1976.0,2011.150810,11.186906,1910.00,2010.000,2015.000,2017.0,2021.00
avaliação,2048.0,4.401123,0.147023,4.20,4.300,4.400,4.5,4.90
num_avaliações,2048.0,573.994629,1376.153171,25.00,58.000,141.000,485.5,32624.00
preço,2048.0,135.242197,272.178314,4.99,31.915,53.625,110.0,3119.08
corpo,2048.0,3.695312,1.566297,0.00,4.000,4.000,5.0,5.00
acidez,2048.0,2.534668,1.038003,0.00,3.000,3.000,3.0,3.00


Valores de string:

In [65]:
df.describe(include='object').T

,count,unique,top,freq
vinícola,2048,480,Vega Sicilia,97
vinho,2048,847,Tinto,56
região,2048,76,Ribera del Duero,541
tipo,1942,21,Ribera Del Duero Red,535
Categoria,2048,5,Vinhos Tintos,1698


## Tratamento e Transformação de dados:

* **Verificando os Valores Faltantes**:

In [66]:
valores_nulos = df.isnull().values.any()

if valores_nulos:
    print("Há valores nulos na tabela.")
else:
    print("Não há valores nulos na tabela.")

Há valores nulos na tabela.


In [67]:
print(f'Verificando valores faltantes : \n\n{df.isna().sum()}')

Verificando valores faltantes : 

vinícola            0
vinho               0
ano                72
avaliação           0
num_avaliações      0
região              0
preço               0
tipo              106
corpo               0
acidez              0
Categoria           0
dtype: int64


* **Verificando os Valores discrepantes**:

In [68]:
# Função para identificar outliers usando o IQR
def detect_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)  # Primeiro quartil
    Q3 = df[column].quantile(0.75)  # Terceiro quartil
    IQR = Q3 - Q1                     # Intervalo Interquartil
    
    lower_bound = Q1 - 1.5 * IQR      # Limite inferior
    upper_bound = Q3 + 1.5 * IQR      # Limite superior
    
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return outliers

In [69]:
# Detectando outliers apenas para colunas numéricas
for col in df.select_dtypes(include=[np.number]).columns:  # Filtra apenas colunas numéricas
    outliers = detect_outliers_iqr(df, col)
    print(f"Outliers na coluna '{col}':\n{outliers}\n")

Outliers na coluna 'ano':
                 vinícola                   vinho     ano  avaliação  \
3            Vega Sicilia                   Unico  1999.0        4.8   
4            Vega Sicilia                   Unico  1996.0        4.8   
5            Vega Sicilia                   Unico  1998.0        4.8   
7            Vega Sicilia                   Unico  1995.0        4.8   
13           Vega Sicilia                   Unico  1970.0        4.8   
...                   ...                     ...     ...        ...   
1777  Marques de Murrieta            Dalmau Rioja  1996.0        4.3   
1778    Marques de Riscal           Rioja Reserva  1964.0        4.3   
1786     Bodegas Faustino          I Gran Reserva  1968.0        4.3   
1858     Bodegas Faustino          I Gran Reserva  1994.0        4.2   
1994             Culebron  Fondillon Gran Reserva  1964.0        4.2   

      num_avaliações            região   preço                  tipo  corpo  \
3               1705  Ribera d

Acima temos uma lista com os valores discrepantes das colunas númericas.

* **Estruturação de Variáveis**:

In [70]:
# Variável alvo
target_variable = 'preço'

# Variáveis inúteis
useless_variables = (
    df_dict
    .query("tipo == 'inútil'")
    .variavel
    .to_list()
)

# Variáveis nominais
nominal_variables = (
    df_dict
    .query("subtipo == 'nominal' and variavel != @target_variable")
    .variavel
    .to_list()
)

# Variáveis ordinais
ordinal_variables = (
    df_dict
    .query("subtipo == 'ordinal' and variavel != @target_variable")
    .variavel
    .to_list()
)

# Variáveis contínuas
continuous_variables = (
    df_dict
    .query("subtipo == 'contínua' and variavel != @target_variable")
    .variavel
    .to_list()
)

# Variáveis discretas
discrete_variables = (
    df_dict
    .query("subtipo == 'discreta' and variavel != @target_variable")
    .variavel
    .to_list()
)

In [82]:
# Exibindo as categorias de variáveis
print("Nominal Variables:", nominal_variables)
print("Ordinal Variables:", ordinal_variables)
print("Continuous Variables:", continuous_variables)
print("Discrete Variables:", discrete_variables)

Nominal Variables: ['vinícola', 'vinho', 'região', 'tipo', 'Categoria']
Ordinal Variables: ['corpo', 'acidez']
Continuous Variables: ['avaliação']
Discrete Variables: ['ano', 'num_avaliações']


O código acima organiza e prepara os dados de forma sistemática para análise ou modelagem. Ele facilita a aplicação de diferentes técnicas a cada tipo de variável, como codificação (para nominais) ou normalização (para contínuas)

* **Separando as variáveis preditoras (x) e a variável alvo (y)** :

In [ ]:
X = df.drop(columns=[target_variable] + useless_variables)
y = df[target_variable]

In [80]:
print("\nX (features):\n", X)


X (features):
                 vinícola            vinho     ano  avaliação  num_avaliações  \
0          Teso La Monja            Tinto  2013.0        4.9              58   
1                 Artadi    Vina El Pison  2018.0        4.9              31   
2           Vega Sicilia            Unico  2009.0        4.8            1793   
3           Vega Sicilia            Unico  1999.0        4.8            1705   
4           Vega Sicilia            Unico  1996.0        4.8            1309   
...                  ...              ...     ...        ...             ...   
2043         Mustiguillo  Finca Terrerazo  2017.0        4.2             390   
2044         Matarromera     Gran Reserva  2011.0        4.2             389   
2045            Sei Solo         Preludio  2016.0        4.2             388   
2046  Vinedos de Paganos       El Puntido  2005.0        4.2             384   
2047   Remirez de Ganuza     Rioja Blanco  2017.0        4.2             421   

                região 

In [81]:
print("\ny (target):\n", y)


y (target):
 0       995.00
1       313.50
2       324.95
3       692.96
4       778.06
         ...  
2043     24.45
2044     64.50
2045     31.63
2046     73.00
2047     29.15
Name: preço, Length: 2048, dtype: float64


* **Pipelines de pré-processamento por tipo de variável** :

In [73]:
# Nominal variables: imputação de valores faltantes e codificação one-hot
nominal_preprocessor = Pipeline(steps=[
    ("missing", SimpleImputer(strategy="most_frequent")),  # Preenchimento com o valor mais frequente
    ("encoding", OneHotEncoder(sparse_output=False))  # Codificação One-Hot
])

# Ordinal variables: imputação de valores faltantes e codificação ordinal
ordinal_preprocessor = Pipeline(steps=[
    ("missing", SimpleImputer(strategy="median")),  # Preenchimento com a mediana
    ("encoding", OrdinalEncoder())  # Codificação ordinal
])

# Continuous variables: imputação de valores faltantes e normalização
continuous_preprocessor = Pipeline(steps=[
    ("missing", SimpleImputer(strategy="mean")),  # Preenchimento com a média
    ("normalization", StandardScaler())  # Normalização
])

# Discrete variables: imputação de valores faltantes e normalização
discrete_preprocessor = Pipeline(steps=[
    ("missing", KNNImputer()),  # Preenchimento baseado em K vizinhos mais próximos
    ("normalization", StandardScaler())  # Normalização
])

Esse código acima organiza pipelines para pré-processar variáveis diferentes (nominais, ordinais, contínuas e discretas). Cada tipo de variável tem necessidades específicas, como tratamento de valores faltantes ou codificação.

In [ ]:
# exibir os pré-processadores criados
print("Nominal Preprocessor:", nominal_preprocessor)
print("Ordinal Preprocessor:", ordinal_preprocessor)
print("Continuous Preprocessor:", continuous_preprocessor)
print("Discrete Preprocessor:", discrete_preprocessor)

Nominal Preprocessor: Pipeline(steps=[('missing', SimpleImputer(strategy='most_frequent')),
                ('encoding', OneHotEncoder(sparse_output=False))])
Ordinal Preprocessor: Pipeline(steps=[('missing', SimpleImputer(strategy='median')),
                ('encoding', OrdinalEncoder())])
Continuous Preprocessor: Pipeline(steps=[('missing', SimpleImputer()),
                ('normalization', StandardScaler())])
Discrete Preprocessor: Pipeline(steps=[('missing', KNNImputer()), ('normalization', StandardScaler())])


* **Selecionando variáveis com base no dados_dict** :

In [ ]:
# Criação do ColumnTransformer para processar cada tipo de variável
preprocessor = ColumnTransformer([
    ("nominal", nominal_preprocessor, nominal_variables),   
    ("ordinal", ordinal_preprocessor, ordinal_variables),   
    ("continuous", continuous_preprocessor, continuous_variables),  
    ("discrete", discrete_preprocessor, discrete_variables), 
])

As variáveis nominais, contínuas, ordinais e discretas são extraídas do dicionário de dados com base no subtipo. Isso assegura que cada tipo de dado seja tratado com o pipeline apropriado.

In [ ]:
print(preprocessor)

ColumnTransformer(transformers=[('nominal',
                                 Pipeline(steps=[('missing',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('encoding',
                                                  OneHotEncoder(sparse_output=False))]),
                                 ['vinícola', 'vinho', 'região', 'tipo',
                                  'Categoria']),
                                ('ordinal',
                                 Pipeline(steps=[('missing',
                                                  SimpleImputer(strategy='median')),
                                                 ('encoding',
                                                  OrdinalEncoder())]),
                                 ['corpo', 'acidez']),
                                ('continuous',
                                 Pipeline(steps=[('missing', SimpleImputer()),
                          

* **Aplicando o preprocessor aos dados** :

In [77]:
X_preprocessed = preprocessor.fit_transform(X)

Acima Treina o pipeline no dataset (fit) e transforma os dados (transform) de acordo com os passos definidos.

In [78]:
print("Dados pré-processados:")
print(X_preprocessed)

Dados pré-processados:
[[ 0.          0.          0.         ...  3.39402046  0.16675855
  -0.37504593]
 [ 0.          0.          0.         ...  3.39402046  0.62045099
  -0.39467063]
 [ 0.          0.          0.         ...  2.71368828 -0.1961954
   0.8860228 ]
 ...
 [ 0.          0.          0.         ... -1.36830481  0.43897402
  -0.13518848]
 [ 0.          0.          0.         ... -1.36830481 -0.55914935
  -0.13809584]
 [ 0.          0.          0.         ... -1.36830481  0.52971251
  -0.11120273]]
